In [2]:
import os
from dotenv import load_dotenv


In [3]:
load_dotenv()
OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')

In [4]:
from langchain.document_loaders import PyPDFLoader

In [5]:
%pwd

'c:\\Users\\Rajmohan\\Desktop\\LLMOPS\\InterviewQuestionCreator\\research'

In [6]:
%cd ..

c:\Users\Rajmohan\Desktop\LLMOPS\InterviewQuestionCreator


In [7]:
file_path = "docs/stats.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

In [8]:
data

[Document(page_content='Importance\nand\nthe\nuse\nof\ncorrelation\nin\nStatistics\nIntroduction\nCorrelation\nis\na\nstatistical\nmeasure\nthat\nexpresses\nthe\nextent\nto\nwhich\ntwo\nvariables\nare\nlinearly\nrelated.\nIt\nis\na\ncommon\ntool\nfor\ndescribing\nsimple\nrelationships\nwithout\nmaking\na\nstatement\nabout\ncause\nand\neffect.\nCorrelation\ncoefficients\nrange\nfrom\n-1\nto\n1,\nwith\na\nvalue\nof\n0\nindicating\nno\nlinear\nrelationship\nbetween\nthe\ntwo\nvariables,\na\nvalue\nof\n1\nindicating\na\nperfect\npositive\nlinear\nrelationship,\nand\na\nvalue\nof\n-1\nindicating\na\nperfect\nnegative\nlinear\nrelationship.\nCorrelation\nis\nimportant\nin\nstatistics\nbecause\nit\ncan\nbe\nused\nto\n1.\nIdentify\nrelationships\nbetween\nvariables:\nCorrelation\ncan\nbe\nused\nto\nidentify\nwhether\nthere\nis\na\nrelationship\nbetween\ntwo\nvariables,\nand\nif\nso,\nwhether\nthe\nrelationship\nis\npositive\nor\nnegative.\nThis\ninformation\ncan\nbe\nuseful\nfor\nunderstanding

In [9]:
len(data)

3

In [10]:
question_gen = ""

for page in data:
    question_gen += page.page_content

In [11]:
question_gen

'Importance\nand\nthe\nuse\nof\ncorrelation\nin\nStatistics\nIntroduction\nCorrelation\nis\na\nstatistical\nmeasure\nthat\nexpresses\nthe\nextent\nto\nwhich\ntwo\nvariables\nare\nlinearly\nrelated.\nIt\nis\na\ncommon\ntool\nfor\ndescribing\nsimple\nrelationships\nwithout\nmaking\na\nstatement\nabout\ncause\nand\neffect.\nCorrelation\ncoefficients\nrange\nfrom\n-1\nto\n1,\nwith\na\nvalue\nof\n0\nindicating\nno\nlinear\nrelationship\nbetween\nthe\ntwo\nvariables,\na\nvalue\nof\n1\nindicating\na\nperfect\npositive\nlinear\nrelationship,\nand\na\nvalue\nof\n-1\nindicating\na\nperfect\nnegative\nlinear\nrelationship.\nCorrelation\nis\nimportant\nin\nstatistics\nbecause\nit\ncan\nbe\nused\nto\n1.\nIdentify\nrelationships\nbetween\nvariables:\nCorrelation\ncan\nbe\nused\nto\nidentify\nwhether\nthere\nis\na\nrelationship\nbetween\ntwo\nvariables,\nand\nif\nso,\nwhether\nthe\nrelationship\nis\npositive\nor\nnegative.\nThis\ninformation\ncan\nbe\nuseful\nfor\nunderstanding\nthe\nrelationships\nb

In [12]:
from langchain.text_splitter import TokenTextSplitter

In [13]:
splitter_ques_gen = TokenTextSplitter(model_name="gpt-3.5-turbo",chunk_size=10000,chunk_overlap=200)

In [14]:
chunk_ques_gen =splitter_ques_gen.split_text(question_gen)

In [15]:
len(chunk_ques_gen)

1

In [16]:
type(chunk_ques_gen[0])

str

In [17]:
from langchain.docstore.document import Document
document_ques_gen = [Document(page_content=t) for t in chunk_ques_gen]

In [18]:
type(document_ques_gen[0])

langchain_core.documents.base.Document

In [19]:
splitter_ans_gen = TokenTextSplitter(model_name="gpt-3.5-turbo",chunk_size=10000,chunk_overlap=200)

In [20]:
chunk_ans_gen =splitter_ans_gen.split_text(question_gen)

In [21]:
len(chunk_ans_gen[0])

4159

In [22]:
document_ans_gen = [Document(page_content=t) for t in chunk_ans_gen]

In [23]:
from langchain.chat_models import ChatOpenAI

In [24]:
llm_ques_gen_pipeline =ChatOpenAI(model='gpt-3.5-turbo',temperature=0.3)

c:\Users\Rajmohan\Anaconda3\envs\interview\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [46]:
Prompt_template ="""

You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""


In [30]:
from langchain.prompts import PromptTemplate

In [48]:
PROMPT_QUESTIONS = PromptTemplate(template=Prompt_template, input_variables=['text'])

In [33]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)

In [34]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [35]:
from langchain.chains.summarize import load_summarize_chain

In [49]:
ques_gen_chain = load_summarize_chain(llm = llm_ques_gen_pipeline, 
                                          chain_type = "refine", 
                                          verbose = True, 
                                          question_prompt=PROMPT_QUESTIONS, 
                                          refine_prompt=REFINE_PROMPT_QUESTIONS)

In [50]:
ques = ques_gen_chain.run(document_ques_gen)

print(ques)

c:\Users\Rajmohan\Anaconda3\envs\interview\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:


You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
Importance
and
the
use
of
correlation
in
Statistics
Introduction
Correlation
is
a
statistical
measure
that
expresses
the
extent
to
which
two
variables
are
linearly
related.
It
is
a
common
tool
for
describing
simple
relationships
without
making
a
statement
about
cause
and
effect.
Correlation
coefficients
range
from
-1
to
1,
with
a
value
of
0
indicating
no
linear
relationship
between
the
two
variables,
a
value
of
1
indicating
a
perfect
positive
linear
relationship,
and
a
value
of
-1
indicating
a
perfect
negative
linear
relationship.
Correlation
is
important
in
statistics
because
it
can
be
used
to
1.
Identify
relationships
between
variables:
Correlation


In [36]:
from langchain.embeddings.openai import OpenAIEmbeddings

In [37]:
embeddings = OpenAIEmbeddings()

c:\Users\Rajmohan\Anaconda3\envs\interview\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [38]:
from langchain.vectorstores import FAISS

In [39]:
vector_store = FAISS.from_documents(document_ans_gen,embeddings)

In [40]:
vector_store

In [43]:
llm_answer_gen =ChatOpenAI(temperature=0.1,model='gpt-3.5-turbo')

In [51]:
ques

'1. What is correlation in statistics and how is it measured?\n2. Explain the importance of correlation in statistics and give examples of how it can be used.\n3. How does correlation differ from causation? Provide an example to illustrate this difference.\n4. In what fields is correlation commonly used and how does it benefit decision-making in those fields?\n5. How can correlation be used to make predictions and develop causal models?\n6. Give examples of real-world scenarios where correlation analysis can be applied in marketing, finance, medicine, and psychology.\n7. Why is it important to understand that correlation does not imply causation? Provide an example to support your explanation.\n8. How does a strong correlation between two variables help in making decisions and improving outcomes in various fields?\n9. Discuss the range of correlation coefficients and what each value indicates in terms of the relationship between variables.\n10. How can correlation analysis help in iden

In [52]:
ques_list = ques.split("\n")

In [53]:
ques_list

['1. What is correlation in statistics and how is it measured?',
 '2. Explain the importance of correlation in statistics and give examples of how it can be used.',
 '3. How does correlation differ from causation? Provide an example to illustrate this difference.',
 '4. In what fields is correlation commonly used and how does it benefit decision-making in those fields?',
 '5. How can correlation be used to make predictions and develop causal models?',
 '6. Give examples of real-world scenarios where correlation analysis can be applied in marketing, finance, medicine, and psychology.',
 '7. Why is it important to understand that correlation does not imply causation? Provide an example to support your explanation.',
 '8. How does a strong correlation between two variables help in making decisions and improving outcomes in various fields?',
 '9. Discuss the range of correlation coefficients and what each value indicates in terms of the relationship between variables.',
 '10. How can corre

In [41]:
from langchain.chains import RetrievalQA

In [54]:
answer_generation_chain = RetrievalQA.from_chain_type(llm=llm_answer_gen, 
                                               chain_type="stuff", 
                                               retriever=vector_store.as_retriever())

In [55]:
# Answer each question and save to a file
for question in ques_list:
    print("Question: ", question)
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")

Question:  1. What is correlation in statistics and how is it measured?
Answer:  Correlation in statistics is a measure that expresses the extent to which two variables are linearly related. It is a statistical tool used to describe simple relationships between variables without making a statement about cause and effect. Correlation coefficients range from -1 to 1, with 0 indicating no linear relationship, 1 indicating a perfect positive linear relationship, and -1 indicating a perfect negative linear relationship.
--------------------------------------------------\n\n
Question:  2. Explain the importance of correlation in statistics and give examples of how it can be used.
Answer:  The importance of correlation in statistics lies in its ability to identify relationships between variables, make predictions, and develop causal models. For example, in business, correlation can help identify relationships between sales and advertising spending to make better marketing decisions. In financ